In [2]:
from src.ddpm.ddpm import DDPM
from src.unet.unet import UNet
from src.data.mnist import get_mnist_loader_and_transform
from torchvision.utils import save_image, make_grid
from src.train import train
import matplotlib.pyplot as plt

import torch.backends
import torch.backends.mps
import os

In [12]:
device = "cuda" if torch.cuda.is_available() else "cpu"
if torch.backends.mps.is_available():
    device = "mps"
    print("USE MPS")
    # os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'

USE MPS


In [21]:
EPOCHS = 10

T = 400

model_with_time = DDPM(T = T, nn_model=UNet(in_channels=1, out_channels=1, T=T+1))
data = get_mnist_loader_and_transform()
_, val_losses = train(
  model=model_with_time,
  optimizer=torch.optim.Adam(params=model_with_time.parameters(), lr=2e-4),
  epochs=EPOCHS,
  device=device,
  train_dataloader=data.train_loader,
  val_dataloader=data.val_loader
)

plt.plot(val_losses, label="with time")

plt.legend()

loss for epoch 0: 0.2100:  11%|█         | 51/469 [01:12<09:55,  1.42s/it] 


KeyboardInterrupt: 

In [ ]:
n_samples = 10
x_t = model_with_time.sample(n_samples=n_samples, size=data.train_dataloader[0][0].shape)
x_t = data.transform_to_pil(x_t)
grid = make_grid(x_t, nrow=10)
save_image(grid, f"epoch_without.png")

cols = 5
rows = (n_samples // cols) + (0 if n_samples % cols == 0 else 1)
fig, axs = plt.subplots(rows, cols, figsize=(3 * cols, 3 * rows))
for i in range(x_t.shape[0]):
    row = i // cols
    axs[row, i % cols].imshow(x_t[i].permute(1,2,0).detach().cpu().numpy(), cmap='gray')